### Метрики оценки качества сборки (финальной)

In [6]:
import cv2
import numpy as np
from skimage.feature import  local_binary_pattern
from scipy.spatial.distance import euclidean
from skimage.feature import graycomatrix, graycoprops

### TSI (LBP, GLCM)

In [7]:
# Параметры LBP
LBP_RADIUS = 1
LBP_N_POINTS = 8 * LBP_RADIUS
LBP_METHOD = 'uniform'


def compute_glcm_features(image):
    image = (image * 255).astype(np.uint8)
    distances = [1]
    angles = [0, np.pi/4, np.pi/2, 3*np.pi/4]
    glcm = graycomatrix(image, distances=distances, angles=angles, levels=256, symmetric=True, normed=True)
    props = ['contrast', 'dissimilarity', 'homogeneity', 'energy', 'correlation', 'ASM']
    features = [np.mean(graycoprops(glcm, p)) for p in props]
    return np.array(features)


def compute_lbp_histogram(image):
    lbp = local_binary_pattern(image, LBP_N_POINTS, LBP_RADIUS, method=LBP_METHOD)
    hist, _ = np.histogram(lbp.ravel(), bins=np.arange(0, LBP_N_POINTS + 3), range=(0, LBP_N_POINTS + 2), density=True)
    return hist


def compute_texture_similarity(feature1, feature2):
    distance = euclidean(feature1, feature2)
    return 1 / (1 + distance)


def compute_texture_similarity_components(img1, img2):
    gray1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY) / 255.0
    gray2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY) / 255.0

    glcm1 = compute_glcm_features(gray1)
    glcm2 = compute_glcm_features(gray2)

    lbp1 = compute_lbp_histogram(gray1)
    lbp2 = compute_lbp_histogram(gray2)

    tsi_glcm = compute_texture_similarity(glcm1, glcm2)
    tsi_lbp = compute_texture_similarity(lbp1, lbp2)

    return tsi_glcm, tsi_lbp


In [8]:
img1 = cv2.imread('pics/1.png')
img2 = cv2.imread('pics/1_.png')

tsi_glcm, tsi_lbp = compute_texture_similarity_components(img1, img2)
print(f"TSI (GLCM): {tsi_glcm:.3f}")
print(f"TSI (LBP):  {tsi_lbp:.3f}")


TSI (GLCM): 0.081
TSI (LBP):  0.864


### Euclidean Matching Accuracy (MA)

In [9]:
def euclidean_matching_accuracy(predicted, ground_truth):
    """
    Compute the Euclidean Matching Accuracy (MA) between predicted and ground truth points.

    Parameters:
        predicted (np.ndarray): Array of shape (N, 2) with predicted (x, y) coordinates.
        ground_truth (np.ndarray): Array of shape (N, 2) with true (x, y) coordinates.

    Returns:
        float: Mean Euclidean distance (MA).
    """
    predicted = np.asarray(predicted)
    ground_truth = np.asarray(ground_truth)

    if predicted.shape != ground_truth.shape:
        raise ValueError("predicted and ground_truth must have the same shape")

    distances = np.linalg.norm(predicted - ground_truth, axis=1)
    mean_accuracy = np.mean(distances)
    return mean_accuracy

Пример использования

In [10]:
# Пример координат
pred_points = np.array([
    [100.5, 200.4],
    [150.1, 300.2],
    [250.0, 400.0]
])

true_points = np.array([
    [100.0, 200.0],
    [150.0, 300.0],
    [251.0, 401.0]
])

ma = euclidean_matching_accuracy(pred_points, true_points)
print(f"Euclidean Matching Accuracy (MA): {ma:.3f} pixels")


Euclidean Matching Accuracy (MA): 0.759 pixels
